# Introduction

Ce notebook présente rapidement la façon de créer un modèle à base de RNN à l'aide de la bibliothèque Keras.

Afin de comprendre tout le pipeline depuis le texte brut, il vaut mieux parcourir le notebook 1-Preprocessing.ipynb.

Les plus impatients pourront quand même commencer ici directement, et implémenter leur premier RNN en utilisant les données déjà préparées :)

Certaines fonctions utilisées ici ont été implémentées dans le fichier rnn_utils.py, n'hésitez pas à consulter celui-ci si vous voulez en savoir plus.

# Imports

In [ ]:
import string
import os

import numpy as np
import pandas as pd

from rnn_utils import pick_char, vocab, index_to_char, char_to_index, one_hot_encode, one_hot_decode

from keras.models import Sequential, load_model
from keras.layers import LSTM, SimpleRNN, Activation, Dense
from tensorflow.keras.utils import plot_model

On définit ici certaines variables globales, notamment la phrase qui va servir de phrase de démarrage pour la génération, ainsi que sa version encodée en one-hot.

# Data

In [ ]:
VOCAB_SIZE = len(vocab)
SEQ_LENGTH = 50
START_SEQ = "mes chers compatriotes nous abordons en ce moment "
ENCODED_START_SEQ = np.expand_dims(one_hot_encode(START_SEQ), 0)

print("Vocabulaire :", vocab)
print("Taille du vocabulaire :", VOCAB_SIZE)
print("Longueur de la séquence d'entrée :", len(START_SEQ))
print("Séquence d'entrée encodée :", ENCODED_START_SEQ.shape)

On charge ensuite nos données d'entraînement issues du notebook de preprocessing.

In [ ]:
inputs = np.load('../../datasets/speeches/speeches_inputs.npy')
targets = np.load('../../datasets/speeches/speeches_targets.npy')

print("Inputs shape :", inputs.shape)
print("Targets shape :", targets.shape)

# Model

Dans Keras, l'une des façons d'implémenter un réseau de neurones est d'utiliser la classe Sequential().

On crée un model Sequential, et ensuite on empile les couches souhaitées grâce à la fonction .add().

On implémente un modèle simple, composée de deux couches :
- Une couche RNN de 16 unités et qui accepte en entrée des matrices de taille (50, 39) issues du preprocessing
- Une couche Dense avec 39 unités et une fonction d'activation softmax, qui nous permet de récupérer en sortie une distribution de probabilités sur les 39 caractères possibles.

In [ ]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(SEQ_LENGTH, VOCAB_SIZE)))
model.add(Dense(39, activation='softmax'))

Le résumé de notre modèle nous permet de voir :
- les dimensions en sortie de chaque couche
- le nombre de poids appris à chaque couche

In [ ]:
model.summary()

In [ ]:
plot_model(model, dpi=120, show_shapes=True)

On compile ensuite le modèle en précisant la fonction de coût et l'algorithme d'optimisation à utiliser.

Puis on lance l'entraînement du modèle pendant 5 itérations, en gardant une partie des données (20%) pour la validation.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(inputs, targets, batch_size=64, epochs=5, validation_split=0.2)

On vérifie que la prédiction du modèle correspond bien à une distribution de probabilités sur 39 valeurs possibles.

In [ ]:
pred = model.predict(ENCODED_START_SEQ)
print("Taille de la matrice prédite pour une séquence unique en input :", pred.shape)
print()
print("Matrice prédite :")
print(pred)
print()
print("Somme des valeurs de la matrice prédite :", pred.sum())

Si on choisit systématiquement la valeur correspondant à la probabilité la plus élevée, on aura systématiquement la même prédiction chaque fois qu'on relance notre modèle avec le même input.

Pour avoir un peu plus de variété, on effectue plutôt un tirage dans la distribution de probabilité. 
Le code suivant montre le résultat de plusieurs tirages successifs, en utilisant la fonction pick_char implémentée dans rnn_utils.py

In [ ]:
for i in range(15):
    chosen_char = pick_char(pred[0], temperature=0.8, do_sample=True)
    print(i+1, "\t - Caractère choisi :", chosen_char, "- Probabilité :", pred[0, char_to_index[chosen_char]])

Il nous suffit alors de répéter les étapes suivantes jusqu'à obtenir un paragraphe de la longueur souhaitée :
- On fournit une séquence en input de longueur 50 (START_SEQ au début)
- On prédit le caractère suivant next_char
- On colle next_char à la fin de l'input précédent, dont on enlève le premier caractère. On obtient alors un nouvel input de taille 50.

In [ ]:
PRED_LENGTH = 500
inp = START_SEQ
result_paragraph = START_SEQ

for i in range(PRED_LENGTH):
    encoded_inp = np.expand_dims(one_hot_encode(inp), 0)
    probas = model.predict(encoded_inp)[0]
    next_char = pick_char(probas, temperature=0.8, do_sample=True)
    result_paragraph = result_paragraph + next_char
    inp = inp[1:] + next_char

In [ ]:
result_paragraph

Bien que loin d'être parfait, le paragraphe prédit montre certaines caractéristiques plutôt encourageantes :
- A partir de caractères, le modèle recrée des "mots" d'une longueur raisonnable
- Il comprend qu'il doit alterner consonnes et voyelles
- Certains mots corrects apparaissent (nation, dans, part, mais...)

Il faut savoir que le modèle qu'on a créé est très peu complexe, et est très insuffisant pour cette tâche. Par ailleurs, le fait de l'avoir entraîné seulement sur 5 itérations ne suffit pas pour obtenir un résultat satisfaisant.

Dans ce qui suit, on fait la même prédiction avec un meilleur modèle, constitué de 2 couches de LSTM (une meilleure variante de RNN), et entrainé pendant 150 itérations sur un nombre plus important de discours avec un vocabulaire plus réduit (sans les caractères numériques)

In [ ]:
better_model = load_model("./256_256_150iter_50seq_punct_0.6732.hdf5")

smaller_index_to_char = {0: ' ', 1: "'", 2: ',', 3: '.', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'j', 14: 'k', 15: 'l', 16: 'm', 17: 'n', 18: 'o', 19: 'p', 20: 'q', 21: 'r', 22: 's', 23: 't', 24: 'u', 25: 'v', 26: 'w', 27: 'x', 28: 'y', 29: 'z'}
smaller_char_to_index = {' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29}

PRED_LENGTH = 500
new_inp = START_SEQ
new_result_paragraph = START_SEQ

for i in range(PRED_LENGTH):
    encoded_inp = np.expand_dims(one_hot_encode(new_inp, char_to_index=smaller_char_to_index), 0)
    probas = better_model.predict(encoded_inp)[0]
    next_char = pick_char(probas, temperature=1.0, do_sample=True, index_to_char=smaller_index_to_char)
    new_result_paragraph = new_result_paragraph + next_char
    new_inp = new_inp[1:] + next_char

In [ ]:
new_result_paragraph